# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [9]:
# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        temp = [s for s in lines[1] if s.isdigit()]
        value = int(''.join(temp))
        self.quantity = value     #un po' articolato ma funziona
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer = lines[-1]
        
        # Salva il prezzo richiesto dal cliente (bid price)
        string = lines[3].split(" ")[-1]
        self.bid_price = float(string[:-1])   #si poteva fare anche in una riga ma è più comprensibile così
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        return self.quantity*ask_price
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        return (self.quantity*ask_price)-(cost_per_km*self.Distance())
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders

orders = []
c = 0
for i in range(1,num_orders+1):
    fileName = str(i)
    testo = open(f"data\{fileName}.txt", "r").read()
    ordineTemp = Order(testo)
    orders.append(ordineTemp)
    #orders[i-1].Details()




In [10]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. 
#Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si 
#otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo 
#                                        di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
def GeneratePossibleProfits():
    possible_profits = []
    for i in range(len(orders)):
        possible_profits.append(0.0)
        try_price = orders[i].bid_price
        for j in range(len(orders)):
            temp = orders[j].Payment(try_price-6) 
            if try_price<=orders[j].bid_price:
                possible_profits[i] += temp
    return possible_profits
    
    
elenco_profitti_possibili = GeneratePossibleProfits()

            
# Determina il profitto maggiore
max_profit = max(elenco_profitti_possibili)

# Determina il prezzo migliore
best_price = orders[elenco_profitti_possibili.index(max_profit)].bid_price

print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.bid_price>best_price]   #modificato perchè prima metteva tutti quelli con profitto positivo, senza considerare che alcuni non sarebbero partiti per via del prezzo fissato
print("Si eseguiranno", len(orders_to_go),"ordini")




Il miglior prezzo è €13.22
Si eseguiranno 193 ordini


In [11]:
#NON CORREGGERE, NON FUNZIONA



# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine 
#sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che 
#porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che
#porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits_delivery = []
for i in range(len(orders)):
    possible_profits_delivery.append(0.0)
    try_price = orders[i].bid_price
    for j in range(len(orders)):
        if orders[j].quantity <100:
            temp = orders[j].Profit(try_price-6,cost_less_100)
            if try_price<=orders[j].bid_price:
                possible_profits_delivery[i] += temp
        else:
            temp = orders[j].Profit(try_price-6,cost_at_least_100)
            if try_price<=orders[j].bid_price:
                possible_profits_delivery[i] += temp

# Determina il profitto maggiore
max_profit = max(possible_profits_delivery)
print(max_profit)

# Determina il prezzo migliore
best_price = orders[possible_profits_delivery.index(max_profit)].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.bid_price>best_price]


indici_ordinati = sorted(range(len(possible_profits_delivery)), key=lambda k: possible_profits_delivery[k])

ordini_in_ordine = [orders[i] for i in indici_ordinati]


#_ = [otg.Details() for otg in orders_to_go]

359166.8432824583
Il miglior prezzo è €13.22
252.70000000000002
2909.6600000000003
3321.2000000000003
2656.96
1696.7
3357.3
498.18000000000006
2021.6000000000001
1631.7200000000003
2967.42
3566.6800000000003
382.66
3140.7000000000003
1797.7800000000002
2656.96
1653.38
2721.94
2223.76
880.84
1747.2400000000002
209.38000000000002
3422.28
397.1
361.00000000000006
2548.6600000000003
823.08
3090.1600000000003
2281.52
2534.2200000000003
974.7
1198.5200000000002
1768.9
3508.9200000000005
2324.84
1126.3200000000002
3559.4600000000005
3443.9400000000005
310.46000000000004
2642.5200000000004
3552.2400000000002
1104.66
2498.1200000000003
1270.72
2072.1400000000003
2765.26
1718.3600000000001
3545.0200000000004
1306.8200000000002
3256.2200000000003
3068.5000000000005
3039.6200000000003
1104.66
324.90000000000003
3032.4
3111.82
1350.14
1176.8600000000001
2656.96
3321.2000000000003
490.96000000000004
2209.32
2238.2000000000003
2317.6200000000003
2462.0200000000004
2635.3
2303.1800000000003
231.040000